In [3]:
import os
import uuid
import requests
import base64
import cloudinary
import cloudinary.uploader
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
# === 配置豆包和cloudinary ===
client = OpenAI(
    base_url="https://ark.cn-beijing.volces.com/api/v3",
    api_key=os.environ.get("OPENAI_API_KEY"),
)

cloudinary.config(
    cloud_name=os.environ.get("CLOUDINARY_CLOUD_NAME"),
    api_key=os.environ.get("CLOUDINARY_API_KEY"),
    api_secret=os.environ.get("CLOUDINARY_API_SECRET"),
    secure=True
)
sys_prompt_text = f"""
你是一个为给图片进行分类打标签的专家，可以根据图片内容，为图片分类并打标签。
下面的<LABEL-INFO></LABEL-INFO>内是标签的具体信息，你需要根据这些标签给图片分类。
标签分为一级标签和二级标签，回答的文本内容格式为一级标签:二级标签，二级标签。
一个一级标签可以对应多个二级标签
场景必须给出
节日主题和特殊活动如果看不出来则不需要给。
季节主题如果不明显则通过人物的穿着冷暖来判断，如果无法分辨是春季还是秋季请都列出。

<LABEL-INFO>
| 一级标签 | 二级标签|
| --- | --- |
| 季节主题 | 春季/夏季/秋季/冬季 |
| 节日主题 | 春节/万圣节/圣诞节/情人节/端午节/中秋节 |
| 特殊活动 | 婚礼/毕业/生日/旅行 |
| 场景 | 家庭/影棚/咖啡馆/图书馆/精品店/教室/CBD/历史街区/天台/地铁站/桥梁/街道/中式园林/公园草地/湖泊/路牌/动物园/迪士尼/漫展/自然/操场 |
| 时间 | 清晨/正午/黄昏/夜晚/黄金时刻 |
| 天气 | 晴天/雾霾/阴天/绵绵小雨/大雨/飘雪/大雪/极光 |
</LABEL-INFO>
"""
# === 上传图片到 Cloudinary 并返回URL ===
def upload_image_to_cloudinary(image_path):
    try:
        upload_result = cloudinary.uploader.upload(image_path)
        return upload_result['secure_url']
    except Exception as e:
        raise Exception(f"上传到Cloudinary失败: {str(e)}")

def image_analysis(filepath,sys_prompt_text):
    
    try:
        image_url = upload_image_to_cloudinary(filepath)

        os.remove(filepath)  # 删除本地文件

        # 调用豆包API
        messages = [{
            "role": "user",
            "content": [
                {"type": "text", "text":sys_prompt_text},
                {"type": "image_url", "image_url": {"url": image_url}}
            ]
        }]

        response = client.chat.completions.create(
            model="doubao-1.5-vision-pro-250328",
            messages=messages
        )
        answer = response.choices[0].message.content
        return answer

    except Exception as e:
        # 出错时也删除临时文件
        if os.path.exists(filepath):
            os.remove(filepath)
        print("图片接口出错：", e)
        return jsonify({"error": str(e)}), 500

filepath = '/Users/lanhaotian/Desktop/IMG_20250524_173009.jpg'
answer = image_analysis(filepath,sys_prompt_text)
print(answer)


场景:家庭


In [1]:
import requests
import base64
import json

class DoubaoMultimodalAPI:
    def __init__(self, api_key, sys_prompt_text):
        
        self.api_key = api_key
        self.sys_prompt_text = sys_prompt_text
        self.api_url = "https://api.doubao.com/v1/multimodal/chat/completions"  # 豆包API的URL
    
    def _get_labels_from_image(self, image_path):
        # 读取并编码图像
        with open(image_path, "rb") as image_file:
            encoded_image = base64.b64encode(image_file.read()).decode('utf-8')
        
        # 构建请求头和请求体
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }
        
        payload = {
            "model": "damo-multimodal",  # 豆包的多模态模型名称
            "messages": [
                {"role": "system", "content": self.sys_prompt_text},
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "请分析这张图片并给出相关标签"},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{encoded_image}"
                            }
                        }
                    ]
                }
            ],
            "temperature": 0.7,
            "max_tokens": 512
        }
        
        # 发送请求到豆包API
        response = requests.post(self.api_url, headers=headers, json=payload)
        
        # 检查响应状态
        if response.status_code == 200:
            result = response.json()
            # 从豆包API响应中提取文本内容
            response_text = result.get("choices", [{}])[0].get("message", {}).get("content", "")
            return self._parse_labels(response_text)
        else:
            # 处理错误情况
            print(f"API请求失败: {response.status_code}, {response.text}")
            return []
    
    def _parse_labels(self, response_text):
        # 解析API返回的标签，根据实际返回格式调整
        # 这里假设返回格式是简单的文本，标签用逗号分隔
        if not response_text:
            return []
        # 示例解析逻辑，根据实际返回格式调整
        labels = [label.strip() for label in response_text.split(',')]
        return labels


In [3]:
# 假设已保存代码到 doubao_api.py 文件
#from doubao_api import DoubaoMultimodalAPI
import os
from dotenv import load_dotenv, find_dotenv
# 1. 配置参数
load_dotenv(find_dotenv())
api_key_env_var = 'OPENAI_API_KEY'
API_KEY = os.getenv(api_key_env_var)  # 替换为实际密钥

SYS_PROMPT = """
你是一个为给图片进行分类打标签的专家，可以根据图片内容，为图片分类并打标签。
下面的<LABEL-INFO></LABEL-INFO>内是标签的具体信息，你需要根据这些标签给图片分类。
标签分为一级标签和二级标签，回答的文本内容格式为一级标签:二级标签，二级标签。
一个一级标签可以对应多个二级标签
场景必须给出
节日主题和特殊活动如果看不出来则不需要给。
季节主题如果不明显则通过人物的穿着冷暖来判断，如果无法分辨是春季还是秋季请都列出。

<LABEL-INFO>
| 一级标签 | 二级标签|
| --- | --- |
| 季节主题 | 春季/夏季/秋季/冬季 |
| 节日主题 | 春节/万圣节/圣诞节/情人节/端午节/中秋节 |
| 特殊活动 | 婚礼/毕业/生日/旅行 |
| 场景 | 家庭/影棚/咖啡馆/图书馆/精品店/教室/CBD/历史街区/天台/地铁站/桥梁/街道/中式园林/公园草地/湖泊/路牌/动物园/迪士尼/漫展/自然/操场 |
| 时间 | 清晨/正午/黄昏/夜晚/黄金时刻 |
| 天气 | 晴天/雾霾/阴天/绵绵小雨/大雨/飘雪/大雪/极光 |
</LABEL-INFO>
"""

# 2. 实例化API类
doubao_api = DoubaoMultimodalAPI(
    api_key=API_KEY,
    sys_prompt_text=SYS_PROMPT
)

# 3. 调用方法提取标签（替换为实际图片路径）
image_path = "/Users/lanhaotian/Desktop/IMG_20250524_173009.jpg"
labels = doubao_api._get_labels_from_image(image_path)

# 4. 处理结果
if labels:
    print(f"提取的标签：{labels}")
else:
    print("未提取到标签或API调用失败")

ConnectionError: HTTPSConnectionPool(host='api.doubao.com', port=443): Max retries exceeded with url: /v1/multimodal/chat/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x106ec6f10>: Failed to resolve 'api.doubao.com' ([Errno 8] nodename nor servname provided, or not known)"))

In [ ]:
import os
import uuid
import requests
import base64
import cloudinary
import cloudinary.uploader
from openai import OpenAI
from dotenv import load_dotenv

client = OpenAI(
    base_url="https://ark.cn-beijing.volces.com/api/v3",
    api_key=os.environ.get("OPENAI_API_KEY"),
)

cloudinary.config(
    cloud_name=os.environ.get("CLOUDINARY_CLOUD_NAME"),
    api_key=os.environ.get("CLOUDINARY_API_KEY"),
    api_secret=os.environ.get("CLOUDINARY_API_SECRET"),
    secure=True
)

# === 上传图片到 Cloudinary 并返回URL ===
def upload_image_to_cloudinary(image_path):
    try:
        upload_result = cloudinary.uploader.upload(image_path)
        return upload_result['secure_url']
    except Exception as e:
        raise Exception(f"上传到Cloudinary失败: {str(e)}")